In [ ]:
from pathlib import Path
import itertools as it

from scipy import stats
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.style.use("seaborn-colorblind")

In [ ]:
method_colours = {"com": "tab:green", "moc": "tab:orange"}
nseeds = 3


In [ ]:
def get_summary(directory, max_seed):

    dfs = []
    for seed in range(max_seed):
        try:
            df = pd.read_csv(f"{directory}/{seed}/summary/main.csv")
            df["seed"] = seed
            dfs.append(df)
        except FileNotFoundError as err:
            print(err)

    main = pd.concat(dfs, axis=0, ignore_index=True)
    main = main.drop_duplicates(
        subset=["nrows", "ncols", "memory", "fitness", "seed"], keep="last"
    )

    return main


In [ ]:
com_dir = "../data/edo/cao_over_matching_final_cost"
com = get_summary(com_dir, nseeds)

com.shape


In [ ]:
moc_dir = "../data/edo/matching_over_cao_final_cost"
moc = get_summary(moc_dir, nseeds)

moc.shape


# Top percentile across all generations


In [ ]:
def get_best_percentiles(data, percentile):

    threshold = np.percentile(data["fitness"], percentile)
    return data[data["fitness"] < threshold]


In [ ]:
com_percentile = get_best_percentiles(com, 1)
moc_percentile = get_best_percentiles(moc, 1)

com_percentile.shape, moc_percentile.shape


In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), dpi=300)

ax.hist(
    com_percentile["fitness"],
    color=method_colours["com"],
    alpha=0.5,
    label="$C_{\mathrm{cao}} - C_{\mathrm{match}}$",
)
ax.hist(
    moc_percentile["fitness"],
    color=method_colours["moc"],
    alpha=0.5,
    label="$C_{\mathrm{match}} - C_{\mathrm{cao}}$",
)

ax.set_xlabel("Fitness")
ax.set_ylabel("Frequency")

ax.legend()

plt.tight_layout()
plt.savefig("../img/edo/fitness.pdf", transparent=True)


In [ ]:
def get_stats(data, root):

    dfs = []
    for _, row in data.astype(int).iterrows():

        seed = row["seed"]
        gen = row["generation"]
        ind = row["individual"]

        df = pd.read_csv(f"{root}/{seed}/data/{gen}/{ind}/main.csv", dtype=object)

        top_percentile_dir = Path(f"{root}/top_percentile/") / str(gen) / str(ind)
        top_percentile_dir.mkdir(parents=True, exist_ok=True)

        df.to_csv(top_percentile_dir / "main.csv", index=False)

        pca = PCA(n_components=1)
        data = pca.fit_transform(df)
        dfs.append(pd.DataFrame(data))

    variances, skews, kurtoses = [], [], []
    lower_deciles, upper_deciles, iqrs = [], [], []
    for df in dfs:
        variances += df.var().tolist()
        skews += stats.skew(df).tolist()
        kurtoses += stats.kurtosis(df).tolist()

        lower_deciles.append(np.percentile(df, 10))
        upper_deciles.append(np.percentile(df, 90))
        iqrs.append(stats.iqr(df))

    return {
        "variance": variances,
        "skewness": skews,
        "kurtosis": kurtoses,
        "lower decile": lower_deciles,
        "upper decile": upper_deciles,
        "interquartile range": iqrs,
    }


In [ ]:
com_stats = get_stats(com_percentile, com_dir)
moc_stats = get_stats(moc_percentile, moc_dir)


In [ ]:
def hist_with_dist_plot(statistic, bins=10, steps=100, filename=None):

    fig, ax = plt.subplots(figsize=(7, 3), dpi=300)

    for values, colour, label in zip(
        [com_stats[statistic], moc_stats[statistic]],
        list(method_colours.values()),
        [r"$C_{\mathrm{cao}} - C_{\mathrm{match}}$", "$C_{\mathrm{match}} - C_{\mathrm{cao}}$"],
    ):
        xs = np.linspace(min(values), max(values), steps)
        kernel = stats.gaussian_kde(values)

        ax.hist(values, bins, color=colour, alpha=0.5, label=label, density=True)
        ax.plot(xs, kernel(xs), color=colour)

    ax.set_xlabel(statistic.capitalize())
    ax.set_ylabel("Density")

    ax.legend()
    plt.tight_layout()

    if filename is not None:
        plt.savefig(filename, transparent=True)


In [ ]:
bins = 25
steps = 200

for stat in (
    "variance",
    "interquartile range",
    "skewness",
    "lower decile",
    "kurtosis",
    "upper decile",
):
    hist_with_dist_plot(stat, bins, steps, f"../img/edo/{stat.replace(' ', '_')}.pdf")


In [ ]:
def pair_plot(statistics, colour, cmap=plt.cm.viridis_r, filename=None):

    nstats = len(statistics)
    fig, axes = plt.subplots(nrows=nstats, ncols=nstats, figsize=(12, 12), dpi=200)

    axes = axes.T
    for i, j in it.product(range(nstats), repeat=2):
        ax = axes[i, j]
        xstat, xvalues = list(statistics.items())[i]
        ystat, yvalues = list(statistics.items())[j]

        if j < i:
            ax.set_axis_off()

        elif i == j:
            ax.hist(xvalues, color=colour, alpha=0.5)

        else:
            nbins = 500
            X, Y = np.mgrid[
                min(xvalues) : max(xvalues) : nbins * 1j,
                min(yvalues) : max(yvalues) : nbins * 1j,
            ]
            positions = np.vstack([X.ravel(), Y.ravel()])
            values = np.vstack([xvalues, yvalues])

            kernel = stats.kde.gaussian_kde(values)
            Z = np.reshape(kernel(positions).T, X.shape)

            ax.contourf(X, Y, Z, cmap=cmap)

        if j == nstats - 1:
            ax.set_xlabel(xstat.capitalize())
        else:
            ax.set_xticklabels([])
        if i == 0:
            ax.set_ylabel(ystat.capitalize())
        else:
            ax.set_yticklabels([])

    plt.tight_layout()
    if filename:
        plt.savefig(filename, transparent=True)


In [ ]:
pair_plot(com_stats, method_colours["com"])


In [ ]:
pair_plot(moc_stats, method_colours["moc"], plt.cm.plasma_r)
